## Analysis plots

In [ ]:
import os
from pathlib import Path

import holoviews as hv

from lisa.trace import Trace
from lisa.analysis.tasks import TaskID
from lisa.platforms.platinfo import PlatformInfo

In [ ]:
# Enable bokeh as default backend, for interactive plots.

# THIS MUST BE DONE AFTER ALL IMPORTS.
# Otherwise there might be issues that lead to
# not displaying plots until hv.extension() is called again.

hv.extension('bokeh')

# Load the trace

In [ ]:
base_path = Path(os.getenv('LISA_HOME'))
doc_path = Path(
    base_path,
    'doc',
    'traces'
)

# Load a trace.dat ftrace binary file, see typical_experiment.ipynb on how to collect one using LISA
trace = Trace(
    doc_path / 'trace.dat',
    # Platform information contain all the knowledge that LISA means about the plaform,
    # such as CPU max frequencies, kernel symbols etc
    plat_info=PlatformInfo.from_yaml_map(doc_path / 'plat_info.yml')
)

# DataFrame libraries
LISA supports two dataframe (table) libraries:
* polars: https://docs.pola.rs/api/python/stable/reference/index.html
* pandas: https://pandas.pydata.org/docs/index.html

Polars is a more modern alternative to pandas and most of the internal machinery of LISA has been moved to polars. At this point, pandas is available for backward compatibility and some internal code still has not been converted, but eventually there will not be any direct dependencies on pandas anymore. Since most dataframe-producing APIs are related to the Trace class, the switch between the two libraries can be achieved at that level:

In [ ]:
import polars as pl

# This creates a view of the trace that will provide polars.LazyFrame dataframes.
# It is also possible to create the trace object for polars directly with
# Trace(..., df_fmt='polars-lazyframe'). The result is the same.
trace = trace.get_view(df_fmt='polars-lazyframe')

List of tasks in the trace, defined as a unique combination of a PID and a task name.

It is also possible to create name-only and pid-only `TaskID` in the analysis APIs 
by assigning None to the other field. 

In [ ]:
trace.ana.tasks.task_ids

In [ ]:
task = TaskID(pid=None, comm='small_2-4')

# Default values for all all analysis methods parameters can be changed on the analysis proxy object. 
# This can de-clutter the code, but the result could be changed after an update in case a plot method gains a parameter. 
ana = trace.ana(
    #tasks=[task],
    #task=task,
)

# Holoviews
All plot methods are implemented using the holoviews library:
https://holoviews.org/getting_started/index.html

For more advanced plot customizations 
https://holoviews.org/user_guide/Applying_Customizations.html

Holoviews is an abstraction layer on top of matplotlib and bokeh (and plotly, but this backend is not supported in LISA). This means that all the styling options are backend specific.

In [ ]:
# Call a plot method that will return a holoviews object.
util_fig = ana.load_tracking.plot_task_signals(task, signals=['util'])
util_fig

# Set a style option
... on the Curve elements of the plot. Style options are backend-specific.

The recommended backend is "bokeh", as it will provide the best interactivity.

In [ ]:
util_fig.options('Curve', color='red')

# Save plot to a file

In [ ]:
path = base_path / 'util.html'
ana.load_tracking.plot_task_signals(task, signals=['util'], filepath=path)
print(path)

# Kernelshark-like plot. 

Using `output='ui'` allows displaying dataframes under the plot,
with scrolling synchronised with the cursor.

**Caveat**: The object created attempts to emulate the holoviews plot API,
but if you get some unexpected exceptions when combining plots, revert to output=None.

**Note**: Dataframes can be costly to create, but most of them are then cached on disk for later use,
so repeated creation of the same plot should be quick.

In [ ]:
activation_fig = ana.tasks.plot_tasks_activation(output='ui')
activation_fig

# Create a Layout

* If you don't want the X axis range to be synchronise, use `.options(shared_axes=False)`.
* `.cols(1)` allows getting the plots on top of eachother, rather than side by side.


In [ ]:
layout = activation_fig + util_fig
layout.cols(1)

# Create an Overlay

It only works well if the axes have the same units, otherwise there will be scaling issues

In [ ]:
capa_fig = ana.load_tracking.plot_task_required_capacity(task=task)
capa_fig * util_fig

# Display custom DataFrame

Any number of dataframes can be linked to the plot. They will each get their marker and tab.

In [ ]:
util_df = ana.load_tracking.df_task_signal(task=task, signal='util')
events_df = ana.notebook.df_all_events()

ana.load_tracking.plot_task_signals(
    task=task,
    signals=['util'],
    link_dataframes={'util': util_df, 'all events': events_df},
    output='ui',
)